In [61]:
from flask import Flask, request, jsonify
import time
import os
import azure.cognitiveservices.speech as speechsdk
import json 
from datetime import datetime 
import subprocess
from pydub.utils import mediainfo
import json
import cv2
import os, shutil
import boto3
from io import StringIO
import sys
import time


class VideoDetect:
    jobId = ''
    rek = boto3.client('rekognition', region_name='ap-south-1')
    sqs = boto3.client('sqs', region_name='ap-south-1')
    sns = boto3.client('sns', region_name='ap-south-1')

    roleArn = ''
    bucket = ''
    video = ''
    startJobId = ''

    sqsQueueUrl = ''
    snsTopicArn = ''
    processType = ''

    def __init__(self, role, bucket, video, segment):
        self.roleArn = role
        self.bucket = bucket
        self.video = video
        self.segment = segment

    def GetSQSMessageSuccess(self):

        jobFound = False
        succeeded = False

        dotLine=0
        while jobFound == False:
            sqsResponse = self.sqs.receive_message(QueueUrl=self.sqsQueueUrl, MessageAttributeNames=['ALL'],
                                                   MaxNumberOfMessages=10)

            if sqsResponse:

                if 'Messages' not in sqsResponse:
                    if dotLine<40:
                        print('.', end='')
                        dotLine=dotLine+1
                    else:
                        print()
                        dotLine=0
                    sys.stdout.flush()
                    time.sleep(5)
                    continue

                for message in sqsResponse['Messages']:
                    notification = json.loads(message['Body'])
                    rekMessage = json.loads(notification['Message'])
                    print(rekMessage['JobId'])
                    print(rekMessage['Status'])
                    if rekMessage['JobId'] == self.startJobId:
                        # print('Matching Job Found:' + rekMessage['JobId'])
                        jobFound = True
                        if (rekMessage['Status']=='SUCCEEDED'):
                            succeeded=True

                        self.sqs.delete_message(QueueUrl=self.sqsQueueUrl,
                                                ReceiptHandle=message['ReceiptHandle'])
                    else:
                        pass
                        # print("Job didn't match:" +
                        #       str(rekMessage['JobId']) + ' : ' + self.startJobId)
                    # Delete the unknown message. Consider sending to dead letter queue
                    self.sqs.delete_message(QueueUrl=self.sqsQueueUrl,
                                            ReceiptHandle=message['ReceiptHandle'])


        return succeeded

    # def StartCelebrityRecognition(self):
    def StartContentModeration(self):
        # response=self.rek.start_label_detection(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
        #                                         NotificationChannel={'RoleArn': self.roleArn, 'SNSTopicArn': self.snsTopicArn})
        # response=self.rek.start_celebrity_recognition(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
        print(self.bucket)
        print(self.video)
        response=self.rek.start_content_moderation(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
                                                   NotificationChannel={'RoleArn': self.roleArn, 'SNSTopicArn': self.snsTopicArn})

        self.startJobId=response['JobId']
        # print('Start Job Id: ' + self.startJobId)

    # def GetCelebrityRecognitionResults(self):
    def GetContentModerationResults(self):
        maxResults = 100
        paginationToken = ''
        # finished = False
        #
        # while finished == False:
        response = self.rek.get_content_moderation(JobId=self.startJobId,
                                                   MaxResults=maxResults,
                                                   NextToken=paginationToken,
                                                   SortBy='TIMESTAMP')

        print('Codec: ' + response['VideoMetadata']['Codec'])
        # print('Duration: ' + str(response['VideoMetadata']['DurationMillis']))
        print('Format: ' + response['VideoMetadata']['Format'])
        print('Frame rate: ' + str(response['VideoMetadata']['FrameRate']))
        print()

        result_list = []

        for moderation_label_info in response['ModerationLabels']:
            moderation_label_dict = {}
            segment_offset_seconds = (self.segment - 1) * 60000
            timestamp = moderation_label_info["Timestamp"]
            timestamp += segment_offset_seconds
            label_name = moderation_label_info["ModerationLabel"]["Name"]
            confidence = moderation_label_info["ModerationLabel"]["Confidence"]

            moderation_label_dict['timestamp'] = timestamp
            moderation_label_dict['Moderated Label Name'] = label_name
            moderation_label_dict['Confidence'] = confidence

            result_list.append(moderation_label_dict)

            print('Timestamp: ', timestamp, '\tModerated Label: ', label_name, "\t Confidence: ", confidence)
        return result_list

    def CreateTopicandQueue(self):

        millis = str(int(round(time.time() * 1000)))

        #Create SNS topic

        snsTopicName="AmazonRekognitionExample" + millis

        topicResponse=self.sns.create_topic(Name=snsTopicName)
        self.snsTopicArn = topicResponse['TopicArn']

        #create SQS queue
        sqsQueueName="AmazonRekognitionQueue" + millis
        self.sqs.create_queue(QueueName=sqsQueueName)
        self.sqsQueueUrl = self.sqs.get_queue_url(QueueName=sqsQueueName)['QueueUrl']

        attribs = self.sqs.get_queue_attributes(QueueUrl=self.sqsQueueUrl,
                                                AttributeNames=['QueueArn'])['Attributes']

        sqsQueueArn = attribs['QueueArn']

        # Subscribe SQS queue to SNS topic
        self.sns.subscribe(
            TopicArn=self.snsTopicArn,
            Protocol='sqs',
            Endpoint=sqsQueueArn)

        #Authorize SNS to write SQS queue
        policy = """{{
  "Version":"2012-10-17",
  "Statement":[
    {{
      "Sid":"MyPolicy",
      "Effect":"Allow",
      "Principal" : {{"AWS" : "*"}},
      "Action":"SQS:SendMessage",
      "Resource": "{}",
      "Condition":{{
        "ArnEquals":{{
          "aws:SourceArn": "{}"
        }}
      }}
    }}
  ]
}}""".format(sqsQueueArn, self.snsTopicArn)

        response = self.sqs.set_queue_attributes(
            QueueUrl = self.sqsQueueUrl,
            Attributes = {
                'Policy' : policy
            })

    def DeleteTopicandQueue(self):
        self.sqs.delete_queue(QueueUrl=self.sqsQueueUrl)
        self.sns.delete_topic(TopicArn=self.snsTopicArn)


def delete_temp_file(file_path):
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))


In [62]:
from moviepy.editor import VideoFileClip


def split_video_into_segments(input_file, segment_duration=60):
    try:
        clip = VideoFileClip(input_file)

        # Calculate the total duration of the input video in seconds
        total_duration = clip.duration

        # Calculate the number of segments required
        num_segments = int(total_duration / segment_duration)

        # Extract the file name and extension from the input_file
        file_name, file_extension = input_file.split('.')

        # Split the video into segments
        for i in range(num_segments):
            start_time = i * segment_duration
            end_time = min((i + 1) * segment_duration, total_duration)
            segment_clip = clip.subclip(start_time, end_time)
            output_file = f"{file_name}_segment_{i + 1}.{file_extension}"
            segment_clip.write_videofile(output_file, codec="libx264")

        # For the last segment (if it's shorter than segment_duration)
        if total_duration % segment_duration > 0:
            start_time = num_segments * segment_duration
            end_time = total_duration
            last_segment_clip = clip.subclip(start_time, end_time)
            output_file = f"{file_name}_segment_{num_segments + 1}.{file_extension}"
            last_segment_clip.write_videofile(output_file, codec="libx264")

        clip.close()
        print("Video splitting completed successfully!")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [63]:
def split_video_and_upload_s3(video_path):
    cwd = os.getcwd()
    listdir_earlier = os.listdir(cwd)
    print(listdir_earlier)
    # ffmpeg_cmd = ['ffmpeg', '-i', video_path,  '-c', 'copy', '-map 0', '-segment_time' , '00:01:00', "-f segment -reset_timestamps 1 output%03d.mp4"]
    # ffmpeg_split_cmd = """ffmpeg -i '""" + video_path + """' -c copy -map 0 -segment_time 00:01:00 -f segment -reset_timestamps 1 output%03d.mp4"""
    ffmpeg_split_cmd = """ffmpeg -i 'video/THOMAS_SHELBY_SMOKING.mp4' -c copy -map 0 -segment_time 00:01:00 -f segment -reset_timestamps 1 output%03d.mp4 > /dump 2>&1"""
    # subprocess.check_output(ffmpeg_cmd, stderr=subprocess.STDOUT)
    # subprocess.run(ffmpeg_split_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    try:
        print(ffmpeg_split_cmd)
        subprocess.check_output(ffmpeg_split_cmd, shell=True, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as e:
        raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))

    listdir_now = os.listdir(cwd)
    print(listdir_now)

    # cmd = 'aws s3 cp D:/Python_scripts/AWS_Scripts/my_function/my_deployment_package.zip s3://alt-cap-logsbucket-12leb47kv5fyr/'

In [64]:
def upload_to_s3(video_path):
    cmd = 'aws s3 cp "' + video_path + '" s3://aws-rekognition-dataset-v2'
    # subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    try:
        subprocess.check_output(cmd, shell=True, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as e:
        raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))
    

In [65]:
def get_videos_with_nomenclature():
    eligible_videos = []
    for filename in os.listdir('D:/Demo_Resources/Moderation_AI/video'):
        if filename.endswith('.mp4') and '_segment_' in filename:
            filename = 'D:/Demo_Resources/Moderation_AI/video/' + str(filename)
            eligible_videos.append(filename)
            upload_to_s3(filename)  # Upload the video to S3 (you need to implement this function)
    return eligible_videos

In [66]:
import re

def extract_segment_number(filename):
    pattern = r'_segment_(\d+)\.mp4'
    match = re.search(pattern, filename)
    if match:
        segment_number = int(match.group(1))
        return segment_number
    else:
        return None

In [67]:
from flask import Flask, request, jsonify
import os
import shutil
from moviepy.editor import VideoFileClip
    
app = Flask(__name__)

@app.route('/upload_video', methods=['POST'])
def upload_video():
    try: 
        save_path = 'D:/Demo_Resources/Moderation_AI/video/'
        # json_path = 'D:/Demo_Resources/Transcription_AI/Transcript_flask/json_file/'
        key = "bca1848b86fd404e9737cd243a3cae8a"
        region = "centralindia"
        lang = "en-IN" # See e.g. https://learn.microsoft.com/en-us/dynamics365/fin-ops-core/dev-itpro/help/language-locale
        # path_in = "09-02-2023 LS-01.wav"
        # path_in = "D:/Python_scripts/speech_to_test_azure/New folder/pm_modi.wav"
        # path_out = "D:/Python_scripts/speech_to_test_azure/srt_fol/pm_modi.json"
        # transcribe(key,region,lang,path_in,path_out)
        # Check if the 'video' key exists in the request files
        if 'video' not in request.files:
            return 'No video file found', 400

        video_file = request.files['video']

        # Save the video to the specified path
        video_file.save(save_path + video_file.filename)
        path_of_video = (save_path + video_file.filename)
        # path_of_wav_file = convert_video_to_wav(path_of_video)
        
        # prefix = os.path.dirname(path_of_video)
        # print(prefix)
        # print(type(prefix))

        split_video_into_segments(path_of_video)
        list_video_segments = get_videos_with_nomenclature()

        final_output = []

        for video_segment in list_video_segments:
        # upload_to_s3(path_of_video)

            roleArn = 'arn:aws:iam::244903904648:role/AmazonRekognitionVideoServiceRole'
            bucket = 'aws-rekognition-dataset-v2'
            # UNCOMMENT THIS FOR SHORT VIDEO
            # video = video_file.filename
            video = video_segment
            video = video.split('/')[-1]
            segment = extract_segment_number(video)

            print('Content Moderation Summary:')

            analyzer = VideoDetect(roleArn, bucket, video, segment)
            analyzer.CreateTopicandQueue()

            output_value_of_srt = None

        # analyzer.StartCelebrityRecognition()
            analyzer.StartContentModeration()
            for i in range(1):
                if analyzer.GetSQSMessageSuccess()==True:
                    # analyzer.GetCelebrityRecognitionResults()
                    output_value_of_srt = analyzer.GetContentModerationResults()
                    print('Right before break!!')
                    final_output.append(output_value_of_srt)
                    break
            analyzer.DeleteTopicandQueue()
        for video_segment in list_video_segments:
            delete_temp_file(video_segment)

        # print(output_value_of_srt)

        
        # split_video_and_upload_s3(path_of_video)
        # output_value_of_srt = None

        # print("path_of_wav:",path_of_wav_file)
        # path_of_json = (json_path + video_file.filename)
        # path_of_json = path_of_json.replace(".mp4",".json")
        # transcribe(key,region,lang,path_of_wav_file,path_of_json)

        
        # list_of_words = convert_json_into_words_by_words_list(path_of_json)
        # path_of_srt = path_of_video.replace("video","srt_file").replace(".mp4",".srt")
        # output_value_of_srt = generate_srt_file(list_of_words,path_of_srt)

        # print(path_of_json)
        # print("path_of_wav_file:",path_of_wav_file)
        # return output_value_of_srt, 200
        return final_output, 200
        # file = request.files['video']
        # print("file:",file)
        # # Specify the local path to save the video
        # save_path = './video/'

        # # Save the video to the specified path
        # file.save(os.path.join(save_path, file.filename))

        # # Get the metadata of the video
        # video_path = os.path.join(save_path, file.filename)
        # video = VideoFileClip(video_path)
        # metadata = {
        #     'filename': file.filename,
        #     'duration': video.duration,
        #     'resolution': f"{video.size[0]}x{video.size[1]}",
        #     'fps': video.fps
        # }

        # # Remove the video file from the local path
        # os.remove(video_path)

        # return jsonify(metadata)
    except Exception as e:
        # Log the exception for debugging purposes
        app.logger.error(f"Exception occurred: {e}")
        return str(e), 500

if __name__ == '__main__':
    app.run()
    # print("="*100)
    # print(a)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


Moviepy - Building video D:/Demo_Resources/Moderation_AI/video/Patal_lok_vid_segment_1.mp4.
MoviePy - Writing audio in Patal_lok_vid_segment_1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video D:/Demo_Resources/Moderation_AI/video/Patal_lok_vid_segment_1.mp4



Moviepy - Done !
Moviepy - video ready D:/Demo_Resources/Moderation_AI/video/Patal_lok_vid_segment_1.mp4
Moviepy - Building video D:/Demo_Resources/Moderation_AI/video/Patal_lok_vid_segment_2.mp4.
MoviePy - Writing audio in Patal_lok_vid_segment_2TEMP_MPY_wvf_snd.mp3


...

MoviePy - Done.
Moviepy - Writing video D:/Demo_Resources/Moderation_AI/video/Patal_lok_vid_segment_4.mp4



Moviepy - Done !
Moviepy - video ready D:/Demo_Resources/Moderation_AI/video/Patal_lok_vid_segment_4.mp4
Moviepy - Building video D:/Demo_Resources/Moderation_AI/video/Patal_lok_vid_segment_5.mp4.
MoviePy - Writing audio in Patal_lok_vid_segment_5TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video D:/Demo_Resources/Moderation_AI/video/Patal_lok_vid_segment_5.mp4



Moviepy - Done !
Moviepy - video ready D:/Demo_Resources/Moderation_AI/video/Patal_lok_vid_segment_5.mp4
Video splitting completed successfully!
Content Moderation Summary:
aws-rekognition-dataset-v2
Patal_lok_vid_segment_1.mp4
........71d76c7f68e663e53f8c036c13b77187f04f7d48341b086c4f6ccf2b538366bb
SUCCEEDED
Codec: h264
Format: QuickTime / MOV
Frame rate: 30.0

Timestamp:  29000 	Moderated Label:  Graphic Violence Or Gore 	 Confidence:  66.58389282226562
Timestamp:  29000 	Moderated Label:  Violence 	 Confidence:  66.833984375
Right before break!!
Content Moderation Summary:
aws-rekognition-dataset-v2
Patal_lok_vid_segment_2.mp4
........4feb960b6b8651d498bfc4e950e4dcf444e5b930b873a653b1145bb06e4ffb23
SUCCEEDED
Codec: h264
Format: QuickTime / MOV
Frame rate: 30.0

Timestamp:  63000 	Moderated Label:  Barechested Male 	 Confidence:  51.089508056640625
Timestamp:  63000 	Moderated Label:  Suggestive 	 Confidence:  51.089508056640625
Timestamp:  68000 	Moderated Label:  Barechested Male 	

127.0.0.1 - - [03/Aug/2023 18:16:26] "POST /upload_video HTTP/1.1" 200 -


In [ ]:
!pip install moviepy